In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import pickle

In [47]:
# 1. DATA LOADING AND LABELING
print("1. Loading Data...")
# Load the datasets
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

1. Loading Data...


In [48]:
# Add a 'label' column to both dataframes
df_fake['label'] = 0  # 0 for fake news
df_true['label'] = 1  # 1 for true news

In [49]:
df_fake

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


In [50]:
df_true

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1


In [51]:
# 2. DATA COMBINATION AND CLEANING

print("2. Combining and Cleaning Data...")
# Combine DataFrames
df_combined = pd.concat([df_fake, df_true]).reset_index(drop=True)


2. Combining and Cleaning Data...


In [52]:
# Remove redundant columns: 'title', 'subject', 'date'. We'll use only 'text'.
df = df_combined.drop(['title', 'subject', 'date'], axis=1)

In [53]:
# Drop any rows with missing values (if any)
df.dropna(inplace=True)

In [54]:
# Separate features (text) and target (label)
X = df['text']
y = df['label']

In [55]:
# 3. SPLITTING DATA
print("3. Splitting Data into Training and Test Sets...")
# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


3. Splitting Data into Training and Test Sets...


In [56]:

print(f"Total Samples: {len(X)}")
print(f"Training Samples: {len(X_train)}")
print(f"Testing Samples: {len(X_test)}")

Total Samples: 44898
Training Samples: 35918
Testing Samples: 8980


In [57]:
# 4. TF-IDF VECTORIZER
print("4. Creating TF-IDF Features...")
# Initialize the TfidfVectorizer
# max_df=0.7 ignores terms that appear in more than 70% of the documents (too common)
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)


4. Creating TF-IDF Features...


In [58]:

# Fit and transform the training set; transform the test set
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

In [59]:
# 5. MODEL TRAINING (Passive Aggressive Classifier)
print("5. Training Passive Aggressive Classifier...")
# Initialize a PassiveAggressiveClassifier
pac = PassiveAggressiveClassifier(max_iter=50)



5. Training Passive Aggressive Classifier...


In [60]:
# Train the model
pac.fit(tfidf_train, y_train)

,C,1.0
,fit_intercept,True
,max_iter,50
,tol,0.001
,early_stopping,False
,validation_fraction,0.1
,n_iter_no_change,5
,shuffle,True
,verbose,0
,loss,'hinge'
,n_jobs,None


In [61]:
# 6. MODEL EVALUATION

print("6. Evaluating Model Performance...")
# Predict on the test set
y_pred = pac.predict(tfidf_test)

6. Evaluating Model Performance...


In [62]:
# Calculate and print accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {round(accuracy*100, 2)}%")


Model Accuracy: 99.49%


In [63]:
# Print the Confusion Matrix for a detailed breakdown
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred, labels=[0, 1]))


Confusion Matrix:
[[4712   21]
 [  25 4222]]


In [64]:
# 7. MODEL PERSISTENCE (SAVING AS .pkl)
print("7. Saving Model Assets...")
# Save the trained model
with open('pac_model.pkl', 'wb') as model_file:
    pickle.dump(pac, model_file)


7. Saving Model Assets...


In [66]:
# Save the fitted TfidfVectorizer
with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(tfidf_vectorizer, vectorizer_file)

print("Trained model ('pac_model.pkl') and TF-IDF vectorizer ('tfidf_vectorizer.pkl') successfully saved.")

Trained model ('pac_model.pkl') and TF-IDF vectorizer ('tfidf_vectorizer.pkl') successfully saved.
